In [1]:
%reset -f

import os
import numpy as np
import pandas as pd
import random
import math
from math import e

import configparser
import itertools

In [11]:
# lectura de datos
    
def load_data():
    """Lee los datos del directorio de trabajo.
    """
    global pathoutput
    global nodos, inv, vu, cartectraf
    global HSCD, PVDT, TNTR, TAMB, PDRE, FCMX, FCPR, HVUT, AVUT
    global P1FVAC1, P1FVAC2, P1FCAR1, P1FCAR2, P1FCAR3, P1FCAR4
    global P3FVAC11, P3FVAC12, P3FVAC21, P3FVAC22, P3FVAC31, P3FVAC32
    global P3FCAR11, P3FCAR12, P3FCAR13, P3FCAR21, P3FCAR22, P3FCAR31, P3FCAR32

    pathoutput = working_dir + 'output/'

    ## definicion del archivo txt donde estan los parametros
    parser = configparser.ConfigParser()
    parser.read(working_dir + 'input/params.txt')

    ## parametros globales
    HSCD = float(parser['CALCULOS']['horas_sobrecarga_dia'])
    PVDT = float(parser['CALCULOS']['perdida_vida_diaria_teorica'])
    TNTR = float(parser['CALCULOS']['temperatura_normal_trafo'])
    TAMB = float(parser['CALCULOS']['temperatura_ambiente'])
    PDRE = float(parser['CALCULOS']['probabilidad_deterioro_reubicacion'])
    FCMX = float(parser['CALCULOS']['factor_carga_maxima'])
    FCPR = float(parser['CALCULOS']['factor_carga_promedio'])

    ## calculos con parametros globales
    HVUT = 24 / PVDT
    AVUT = HVUT / (24*365) 

    ##
    ## Resoluciones 818 y 819
    ##

    ##    Transformadores monofasicos - Perdidas en vacio
    ##
    P1FVAC1 = float(parser['RES818819']['Par_1f_vac_1'])
    P1FVAC2 = float(parser['RES818819']['Par_1f_vac_2'])

    ##
    ##    Transformadores monofasicos - Perdidas con carga
    ##        
    P1FCAR1 = float(parser['RES818819']['Par_1f_car_1'])
    P1FCAR2 = float(parser['RES818819']['Par_1f_car_2'])
    P1FCAR3 = float(parser['RES818819']['Par_1f_car_3'])
    P1FCAR4 = float(parser['RES818819']['Par_1f_car_4'])

    ##
    ##    Transformadores trifasicos -- Perdidas en vacio
    ##
    P3FVAC11 = float(parser['RES818819']['Par_3f_vac_11'])
    P3FVAC12 = float(parser['RES818819']['Par_3f_vac_12'])
    P3FVAC21 = float(parser['RES818819']['Par_3f_vac_21'])
    P3FVAC22 = float(parser['RES818819']['Par_3f_vac_22'])
    P3FVAC31 = float(parser['RES818819']['Par_3f_vac_31'])
    P3FVAC32 = float(parser['RES818819']['Par_3f_vac_32'])

    ##
    ##    Transformadores trifasicos - Perdidas con carga
    ##        
    P3FCAR11 = float(parser['RES818819']['Par_3f_car_11'])
    P3FCAR12 = float(parser['RES818819']['Par_3f_car_12'])
    P3FCAR13 = float(parser['RES818819']['Par_3f_car_13'])
    P3FCAR21 = float(parser['RES818819']['Par_3f_car_21'])
    P3FCAR22 = float(parser['RES818819']['Par_3f_car_22'])
    P3FCAR31 = float(parser['RES818819']['Par_3f_car_31'])
    P3FCAR32 = float(parser['RES818819']['Par_3f_car_32'])

    ##
    ## tablas de datos
    ##
    nodos = pd.read_csv(working_dir + "input/nodos.csv", sep=',', decimal='.')
    inv = pd.read_csv(working_dir + "input/inventario_transformadores.csv", sep=',', decimal='.')
    cartectraf = pd.read_csv(working_dir + "input/carac_tecn_transf.csv", sep=',', decimal='.')
    vu = pd.read_csv(working_dir + "input/vida_util.csv", sep=',', decimal='.')

    ##
    ## nombres de las columnas de las tablas de datos
    ##
    nodos.columns = ['id_n', 'id_n_Internexa','lat','lon','tension','cpro_n','cmax_n','cremcreg','dmda_n','cens','cred','tusu','pkwh_n']
    inv.columns = ['id_t', 'id_t_Internexa','fab','fase_t','tais','capa_t','vprim','vsecu','ffab','anus','viut_t','id_n_Internexa','tacr_t','creu_t','finst']
    vu.columns = ['tgrc', 'fase_t','lipo','lspo','cpre','dura','cpor','tmpc','tmac']
    cartectraf.columns = ['fase_t', 'capa_t','cnue_t']

    ## 
    ## adecuacion de las tablas para facilidad en calculos
    ##
    inv = inv.merge(nodos[['id_n','id_n_Internexa']], on = 'id_n_Internexa',how = 'left')

    ## calcular carga maxima y carga promedio de los nodos
    nodos['cpro_n'] = nodos['dmda_n'] / 30 * FCPR
    nodos['cmax_n'] = nodos['dmda_n'] / 30 * FCMX

    ## indicar grupo del trafo para calculo de las perdidas de transformacion
    inv['grpt_t'] = 1
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpt_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 800), 'grpt_t'] = 3

    ## indicar grupo del trafo para calculo de las perdidas de vida util
    inv['grpv_t'] = 1
    inv.loc[(inv.fase_t == 1) & (inv.capa_t > 50), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 500), 'grpv_t'] = 3

    ## calcular vida util restante del trafo en meses
    inv['viut_t'] = AVUT
    inv['viur_t'] = (inv.viut_t - inv.anus) * 12
    inv.loc[inv.viur_t < 0, 'viur_t'] = 1

    ## indicar grupo de vida util
    vu['grpv_t'] = 1
    vu.loc[(vu.fase_t == 1) & (vu.lipo >= 50), 'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 150),'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 500), 'grpv_t'] = 3

    ## armar keys para busquedas
    cartectraf['faca'] = cartectraf.fase_t.map(str) + "-" + cartectraf.capa_t.map(str)
    vu['tfcg'] = vu.tgrc.map(str) + "-" + vu.fase_t.map(str) + "-" + vu.cpre.map(str) + "-" + vu['grpv_t'].map(str)


In [3]:
# funciones core

def parnd(id_n):
    """Obtiene los parametros de un nodo.

    Args:
        id_n (int): id del nodo.

    Returns:
        cmax_n (float): carga maxima que soporta el nodo.
        cpro_n (float): carga promedio que soporta el nodo.
        pkwh_n (float): precio por kWh de la electricidad que sirve el nodo.

    """
    cmax_n = float(nodos[nodos.id_n == id_n]['cmax_n'])
    cpro_n = float(nodos[nodos.id_n == id_n]['cpro_n'])    
    pkwh_n = float(nodos[nodos.id_n == id_n]['pkwh_n']) 
    return (cmax_n,cpro_n,pkwh_n)


def partf(id_t):
    """Obtiene los parametros de un trafo.

    Args:
        id_t (int): id del trafo.

    Returns:
        capa_t (float): capacidad del trafo.
        fase_t (int): numero de fases del trafo.
        viut_t (int): vida utuil teorica del trafo.
        nodo_t (int): nodo al que se encuentra asociado el trafo.
        creu_t (float): costo de la actividad de reubicacion del trafo.
        viur_t (float): vida util restante del trafo.
        grpt_t (int): grupo al que pertenece el trafo para el calculo de las perdidas de transformacion.
        grpv_t (int): grupo al que pertenece el trafo para el calculo de las perdidas de vida util.
        faca_t (str): key fase-capacidad.
        cnue_t (int): precio por kWh de la electricidad que sirve el nodo.

    """
    capa_t = float(inv[inv.id_t == id_t]['capa_t'])
    fase_t = int(inv[inv.id_t == id_t]['fase_t'])
    viut_t = int(inv[inv.id_t == id_t]['viut_t'])
    nodo_t = int(inv[inv.id_t == id_t]['id_n'])
    creu_t = float(inv[inv.id_t == id_t]['creu_t'])
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    grpt_t = int(inv[inv.id_t == id_t]['grpt_t'])
    grpv_t = int(inv[inv.id_t == id_t]['grpv_t'])
    faca_t = str(fase_t) + '-' + str(capa_t)
    cnue_t = int(cartectraf[cartectraf.faca == faca_t]['cnue_t'])
    return (capa_t, fase_t, viut_t, nodo_t, creu_t, viur_t, grpt_t, grpv_t, faca_t, cnue_t)


def cospt(id_n, id_t):
    """Calcula los costos de las perdidas de transformacion de un par nodo*trafo.

    Args:
        id_n (int): id del nodo.
        id_t (int): id del trafo.

    Returns:
        cpt_nt (float): costos de perdidas de transformacion en pesos.

    """
    if id_n == 999999:
        cpt_nt = 0; cpfe_nt = 0; cpcu_nt = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnd(id_n)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,grpv_t,faca_t,cnue_t = partf(id_t)
        futi_nt = cmax_n / capa_t

        # calcular perdidas nominales en vacio y perdidas nominales con carga en funcion de las fases y el grupo en la fase
        if fase_t == 1:
            pnvac = P1FVAC1 * capa_t ** P1FVAC2
            pncar = P1FCAR1 * capa_t ** 3 + P1FCAR2 * capa_t ** 2 + P1FCAR3 * capa_t + P1FCAR4
        else:
            if grpt_t == 1:
                pnvac = P3FVAC11 * capa_t ** P3FVAC12
                pncar = P3FCAR11 * capa_t ** 2 + P3FCAR12 * capa_t + P3FCAR13
            if grpt_t == 2:
                pnvac = P3FVAC21 * capa_t ** P3FVAC22
                pncar = P3FCAR21 * capa_t + P3FCAR22
            if grpt_t == 3:
                pnvac = P3FVAC31 * capa_t ** P3FVAC32
                pncar = P3FCAR31 * capa_t + P3FCAR32

        # calcular las perdidas en hierro y cobre en unidades W 
        pfeW = pnvac
        pcuW = pncar * futi_nt ** 2
        ptrW = pfeW + pcuW

        # monetizacion de las perdidas
        cpfe_nt = pfeW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
        cpcu_nt = pcuW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
        cpt_nt = ptrW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
    return (cpt_nt, cpfe_nt, cpcu_nt)


def cosvu(id_n, id_t):
    """Calcula los costos de las perdidas de vida util de un par nodo*trafo

    Args:
        id_n (int): id del nodo.
        id_t (int): id del trafo.

    Returns:
        cpt_vu (float): costos de perdidas de vida util en pesos.

    """
    if id_n == 999999:
        cvu_nt = 0; viur_r = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnd(id_n)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,grpv_t,faca_t,cnue_t = partf(id_t)

        # calcular la carga precedente y el factor de utilizacion
        cpre_nt = cpro_n / capa_t
        futi_nt = cmax_n / capa_t

        # aproximar la carga precedente a los valores de la norma GTC50
        if cpre_nt < ((0.5 + 0.75) / 2):
            cpre_nt = 0.5
        else:
            if cpre_nt < ((0.75 + 0.9) / 2):
                cpre_nt = 0.75
            else:
                cpre_nt = 0.9

        # calcular el porcentaje diario de perdida de vida util real en porcentaje
        key = str(int(TAMB)) + '-' + str(fase_t) + '-' + str(cpre_nt) + '-' + str(grpv_t)
        theta = temperPC(futi_nt,key)
        fevej = (HSCD / 24) * (e**(15000/383 - 15000/(theta + 273))-1)
        pvdr = PVDT * (1 + fevej)

        # valorar perdida de vida util restante en pesos, durante lo que queda de vida util del trafo en el nodo 
        cvu_nt = viur_t * 30 * cnue_t * pvdr / viur_t
        viur_r = 12 / (pvdr * 365)
    return (cvu_nt, viur_r)


def temperPC(futi_nt, key):
    """Calcula la temperatura del punto mas caliente dado un factor de utilizacion

    Args:
        futi_nt (float): factor de utilizacion.
        key (int): key TAMB - fase_t - cpre_nt - grpv_t.

    Returns:
        theta (int): temperatura del punto mas caliente.

    """
    tabvu = vu.loc[(vu.tfcg == str(key)) & (vu.dura <= HSCD)]
    ncargas = tabvu.shape[0]
    carga=futi_nt * 100
    theta=0
    if carga < tabvu['cpor'].min(): theta = TNTR
    if carga >= tabvu['cpor'].max(): theta = tabvu['tmpc'].max()
    if theta == 0:
        tabvu = tabvu.sort_values(['cpor'],ascending=[False])
        for index, row in tabvu.iterrows():
            if carga <= row['cpor']:
                theta = row['tmpc']
                break
    return theta 


def costopermtf(id_t):
    """Calcula los costos de permutacion del trafo

    Args:
        id_t (int): id del trafo.

    Returns:
        cperm_t (float): costo de permutacion del trafo.

    """
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    cperm_t = float(inv[inv.id_t == id_t]['creu_t']) / viur_t
    return cperm_t


def costodetetf(id_t):
    """Calcula los costos de deterioro de un trafo por reubicacion.

    Args:
        id_t (int): id del trafo.

    Returns:
        cdete_t (float): costo de deterioro del trafo.

    """
    cnue_t = partf(id_t)[9]
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    cdete_t = cnue_t * PDRE / viur_t
    return cdete_t

In [4]:
def bodega():
    """Calcula la bodega.
    """
    global bodini, bodfin
    
    bodini = solini[solini.id_n == 999999]
    bodfin = solfin[solfin.id_n == 999999]

In [5]:
def fitness(sol):
    
    solini = pd.DataFrame(columns=['id_n','id_t','cpt','cpfe','cpcu','cvu','viur_r','coper','cperm','cdete','eval','perm','bloq'])

    for index, row in sol.iterrows():
        id_n = int(row['id_n'])
        id_t = int(row['id_t'])

        # calculos de costos para los nodos reales y en bodega
        cpt_nt = 0; cpfe_nt = 0; cpcu_nt = 0; cvu_nt = 0; viur_r = 0
        if id_n != 999999: 
            cpt_nt, cpfe_nt, cpcu_nt = cospt (id_n,id_t)
            cvu_nt, viur_r = cosvu (id_n,id_t)
        coper_nt = cpt_nt + cvu_nt

        # almacenar el costo para ese arreglo en particular
        solini.loc[index]= [id_n,id_t,cpt_nt,cpfe_nt,cpcu_nt,cvu_nt,viur_r,coper_nt,0,0,0,0,0]

    # escribir la solucion inicial y calcular su costo
    ctsolini = solini['coper'].sum()

    return ctsolini

In [13]:
def listasoluciones():
    """Guarda el total exhaustivo de las posibles soluciones - configuraciones de red 
    """
    
    global nnd, solexh, tsolexh, tfenev

    ntf = inv.shape[0]
    nnd = nodos.shape[0]
    solexh = list(itertools.permutations(range(1, (ntf+1)),(nnd-1)))
    tsolexh = len(solexh)
    solexh = np.array(solexh)
    solexh = pd.DataFrame(solexh.reshape(tsolexh,nnd-1))
    
    # solucion en evaluacion: inicialmente es una matriz con el id de todos los trafos
    tfenev = pd.DataFrame(index=range(ntf))
    tfenev['id_t'] = range(1,ntf+1)

In [ ]:
#######Evaluar todas las posibles soluciones

In [14]:
def GenerarUnaSol2(i):
    """Evalua una solucion semilla de la red con sus costos asociados.
    """
        
    # matriz con el id de los nodos asignados
    ndenev = pd.DataFrame(solexh.iloc[i,:])
    ndenev.columns=['id_t']
    ndenev['id_n'] = range(1,nnd)

    # solucion a evaluar
    sol = tfenev.merge(ndenev, on = 'id_t',how = 'left')
    sol['id_n']=sol['id_n'].fillna(999999)

    return sol

In [15]:
def CalcularCostoPerm(sol):
    sol['resta'] =sol['id_n'] - solori['id_n']
    sol['cperm'] = 0
    sol.loc[(sol.resta != 0), 'cperm'] = sol.loc[(sol.resta != 0), 'id_t'].apply(lambda x: costopermtf(x))
    return sol['cperm'].sum()  

In [16]:
def CalcularCostoDete(sol):
    sol['resta'] =sol['id_n'] - solori['id_n']
    sol['cdete'] = 0
    sol.loc[(sol.resta != 0), 'cdete'] = sol.loc[(sol.resta != 0), 'id_t'].apply(lambda x: costodetetf(x))
    return sol['cdete'].sum()

In [19]:
working_dir="../Tests/Test4/"
load_data()

solori = pd.concat([inv['id_t'],inv['id_n'],], axis=1, sort=False)

# armar el total de posibles soluciones
listasoluciones()
pobexh = []
for i in range (tsolexh):
    pobexh.append(GenerarUnaSol2(i))

In [21]:
# calcular el costo de operacion de las posibles soluciones   
import operator
COPobExh = {}
for i in range(len(pobexh)):
    COPobExh[i] = fitness(pobexh[i])
COPobExhOrd = sorted(COPobExh.items(), key = operator.itemgetter(1), reverse=False)


NameError: name 'operator' is not defined

In [22]:
import operator
COPobExhOrd = sorted(COPobExh.items(), key = operator.itemgetter(1), reverse=False)

In [24]:
#calcular los costos de movimientos de las posibles soluciones
CPPobExh = {}
CDPobExh = {}
for i in range(len(pobexh)):
    CPPobExh[i] = CalcularCostoPerm(pobexh[i])
    CDPobExh[i] = CalcularCostoDete(pobexh[i])
CPPobExhOrd = sorted(CPPobExh.items(), key = operator.itemgetter(1), reverse=False)
CDPobExhOrd = sorted(CDPobExh.items(), key = operator.itemgetter(1), reverse=False)

In [25]:
COSol = np.array(COPobExhOrd)
COSol = pd.DataFrame(COSol.reshape(len(COPobExhOrd),2))

CPSol = np.array(CPPobExhOrd)
CPSol = pd.DataFrame(CPSol.reshape(len(CPPobExhOrd),2))

CDSol = np.array(CDPobExhOrd)
CDSol = pd.DataFrame(CDSol.reshape(len(CDPobExhOrd),2))

CTotal = COSol.merge(CPSol[[0,1]], on = 0,how = 'left')
CTotal = CTotal.merge(CDSol[[0,1]], on = 0,how = 'left')
CTotal.columns = ['sol','coper','cperm','cdeter']
CTotal['ctotal'] = CTotal['coper'] + CTotal['cperm'] + CTotal['cdeter']
CTotal = CTotal.sort_values(['ctotal'],ascending=[True])
mejorsol = int(CTotal.iloc[0,0])
sol = pobexh[mejorsol]

print(CTotal.head())
print(sol)
print(solori)


        sol          coper         cperm        cdeter         ctotal
377  2243.0  675957.277189  11087.467816   9968.983061  697013.728066
376  2242.0  675957.277189  13122.796792  12170.267740  701250.341721
375  2241.0  675957.277189  13122.796792  12170.267740  701250.341721
480     8.0  688526.751038   7452.590777   6037.727116  702017.068931
490  1520.0  688526.751038  13122.796792  12170.267740  713819.815570
   id_t      id_n     resta        cperm        cdete
0     1  999999.0  999998.0  5670.206015  6132.540624
1     2       2.0       0.0     0.000000     0.000000
2     3       3.0       0.0     0.000000     0.000000
3     4  999999.0  999995.0  2334.654301  2525.016253
4     5       5.0       0.0     0.000000     0.000000
5     6       4.0 -999995.0  1541.303750   410.205250
6     7       1.0 -999998.0  1541.303750   901.220934
   id_t    id_n
0     1       1
1     2       2
2     3       3
3     4       4
4     5       5
5     6  999999
6     7  999999
